In [ ]:
!pip install transformers
!pip install imblearn
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from transformers import TFBertForSequenceClassification, BertTokenizer, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EvalPrediction

from torch.utils.data import Dataset
import os

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
import tensorflow as tf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Twitch_Colab/twitch_messages.csv')
print(df.index)
label_counts = df['label'].value_counts()
print(label_counts)

RangeIndex(start=0, stop=4000, step=1)
Irrelevant to Streamer    2555
Relevant to Streamer      1445
Name: label, dtype: int64


In [ ]:
import string
import re

#model_checkpoint = "albert-base-v2"

# Which models can we use?
# 1. Bert, 2. Roberta, 3. Bart,
# Create a directory to save the models
os.makedirs("saved_models", exist_ok=True)

def apply_sampling(X_train,y_train, which_sample):
  if which_sample == 0:
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    return X_train_resampled, y_train_resampled
  elif which_sample == 1:
    tomek = TomekLinks()
    X_train_resampled_tomek, y_train_resampled_tomek = tomek.fit_resample(X_train, y_train)
    return X_train_resampled_tomek, y_train_resampled_tomek
  elif which_sample == 2:
    ncl = NeighbourhoodCleaningRule()
    X_train_resampled_tomek_ncl, y_train_resampled_tomek_ncl = ncl.fit_resample(X_train,
                                                                                y_train)
    return X_train_resampled_tomek_ncl, y_train_resampled_tomek_ncl

  else:
    print("specify the sampling technique")





# Load the dataset and preprocess it
def load_and_preprocess_data():
    # Load the IMDB movie reviews dataset
    #imdb_dataset = load_dataset("imdb")
    df = pd.read_csv('/content/drive/MyDrive/Twitch_Colab/twitch_messages.csv')
    # df2 = pd.read_csv('/content/drive/MyDrive/Twitch_Colab/data.csv')
    #1. replace @name with @author, if any
    unique_values = df['StreamerName'].unique().tolist()

    df['replaced_author'] = df['message']
    #display how many?

    unique_values = df['StreamerName'].unique().tolist()
    for item in unique_values:
        df['replaced_author'] = df['replaced_author'].str.replace(item, 'author')
    unique_values = [x.lower() for x in unique_values]
    for item in unique_values:
        df['replaced_author'] = df['replaced_author'].str.replace(item.lower(), 'author')


    #print("author name was replaced with @author {0} times".format(df['replaced_author'].str.count('author').sum()))
    # 2. Remove all spaces on message itself, without replacing author name
    df['no_spaces'] = df['message'].str.replace(r'\s+', ' ')
    # 3. Remove punctuation, maybe we should not remove it? since some are about questions?
    df['no_punctuation'] =  df['no_spaces'].str.replace('[{}]'.format(string.punctuation), '')

    #df['stop_words'] = df['messages'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords.words('english')]))

    new_df = pd.DataFrame({
        'video_id': df['VidID'],
        'streamer': df['StreamerName'],
        'message': df['message'],
        'replaced_author': df['replaced_author'],
        'no_spaces': df['no_spaces'],
        'no_punctuation':  df['no_punctuation'],
        'label': df['Mohammad_Code']
    })

    # 4. Remove duplicate messages
    new_df = new_df.drop_duplicates(subset=['no_spaces'],keep='first')

    # 5. make it balanced, simple 
    new_df = new_df.dropna()
    new_df['replaced_author'] = new_df['replaced_author'].apply(lambda x: x.replace("@", "@user") if isinstance(x, str) and "@" in x and "@author" not in x else x)

    pattern = r"@\w+"
    for i, row in new_df.iterrows():
        message = row['replaced_author']
        if isinstance(message, str) and "@user" in message:
            # replace any occurrence of "@userstring" with "@user"
            message = re.sub(pattern, "@user", message)
            #print(message)
            # strip "user" from "@user"
            #message = message.replace("@user", "@")
            # update the message in the DataFrame
            df.loc[i, 'replaced_author'] = message

    #label_counts = new_df['label'].value_counts()
    print("before sampling")
    count_rel = (new_df['label'] == 0).sum()
    count_irr = (new_df['label'] == 1).sum()
    print("we have {0} relevant to streamer messages, and {1} irrelevant to streamer messages".format(count_rel, count_irr))
    # Sampling
    # g = new_df.groupby('label')
    # new_df = g.apply(lambda x: x.sample(g.size().min(), random_state=1))


    print("author name was replaced with @author {0} times".format(df['replaced_author'].str.count('@author').sum()))
    print("user name was replaced with @user {0} times".format(df['replaced_author'].str.count('@user').sum()))

    label_counts = new_df['label'].value_counts()
    #print("sampling", label_counts)
    
    # Combine the 'train' and 'test' splits into a single dataset
    # We can change the the pre-processing here
    dataset = {
        'text': new_df['no_spaces'].tolist(),
        'labels': new_df['label'].tolist()
    }

    # print(len(dataset["labels"]))
    # dataset = {
    #   'text': df2['text'].tolist(),
    #   'labels': df2['label'].tolist()
    #   }
    print("After Sampling")
    count_rel = (new_df['label'] == 0).sum()
    count_irr = (new_df['label'] == 1).sum()
    print("we have {0} relevant to streamer messages, and {1} irrelevant to streamer messages".format(count_rel, count_irr))

    return dataset

# Custom dataset class
class BinaryClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Function to encode the dataset
def encode_dataset(tokenizer, texts, labels, max_length=512):
    encoded_data = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return BinaryClassificationDataset(encoded_data, labels)


def predict(text, model, tokenizer):
    # Preprocess the input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')

    # Move the model to the same device as the input tensors
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class (0 or 1)
    predicted_class = torch.argmax(outputs.logits, dim=-1).item()

    return predicted_class

# load_and_preprocess_data()


<ipython-input-5-be4ea1becc2a>:52: FutureWarning: The default value of regex will change from True to False in a future version.
  df['no_spaces'] = df['replaced_author'].str.replace(r'\s+', ' ')
<ipython-input-5-be4ea1becc2a>:54: FutureWarning: The default value of regex will change from True to False in a future version.
  df['no_punctuation'] =  df['message'].str.replace('[{}]'.format(string.punctuation), '')


0                                                  ohhhhh
1             If i set my legs on fire will i run faster?
2                                              aaayyyeeee
3       entobiBugLove entobiBugLove entobiBugLove ento...
4          @user thank you for the gift sub!! gopiraHeart
                              ...                        
3993                                      just subscribed
3994       you can also do go doc strings | grep contains
3995                                                   Yo
3997                               PORQUEEEEEEE MARIAAAA?
3999            I get so stoked when I see you streaming.
Name: replaced_author, Length: 3404, dtype: object
before sampling
we have 1413 relevant to streamer messages, and 1991 irrelevant to streamer messages
author name was replaced with @author 36 times
user name was replaced with @user 99 times
After Sampling
we have 1413 relevant to streamer messages, and 1991 irrelevant to streamer messages


{'text': ['ohhhhh',
  'If i set my legs on fire will i run faster',
  'aaayyyeeee',
  'entobiBugLove entobiBugLove entobiBugLove entobiBugLove entobiBugLove entobiNasiPog entobiBugLove entobiBugLove entobiBugLove entobiBugLove entobiBugLove entobiBugLove entobiBugLove',
  'leor0111 thank you for the gift sub gopiraHeart',
  'gopiraSip',
  'Hello how are you today',
  'Enderscram game dev is a small world thats why',
  'anyone know of rose of starcross  am i interrupting something ',
  'do you support lgbtq',
  'gopiraDoggers',
  'I love that it can be both a chemistry and guacamole pun',
  'so Thor what about that secret beatem up you were working on last year is it finished is it published I lost a couple streams so I didnt got any news about it',
  'NAME',
  'choose  the bottom yes',
  'I support everyone but Thor',
  'nice',
  'LUL',
  'Thank you blumberquack',
  'updates',
  'I forgot how incredible that animation is LUL',
  'hey GoPirateSoftware whats the update on the game dev ca

In [ ]:
# Define which model will we be using
#done the first one

models = ["setfit-distilbert-user-intent"] 
models = ["albert-base-v2", "bert-base-uncased" , "roberta-base", "distilbert-base-uncased"]
# models = ["bert-base-uncased"]
sampling_technique = 0
print("Unprocessed")
for m in models:
  tokenizer = AutoTokenizer.from_pretrained(m)

  # Load and preprocess the dataset
  dataset = load_and_preprocess_data()

  # Initialize StratifiedKFold with 10 folds
  n_splits = 5
  kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

  # Collect the evaluation metrics across all folds
  accuracy_list = []
  precision_list = []
  recall_list = []
  f1_list = []
  smote = SMOTE(random_state=42)

  for train_index, val_index in kfold.split(dataset['text'], dataset['labels']):
      train_texts = [dataset['text'][i] for i in train_index]
      train_labels = [dataset['labels'][i] for i in train_index]
      # train_texts, train_labels = apply_sampling(train_texts, train_labels, sampling_technique)
      print(len(val_index))
      print(len(train_index))


      val_texts = [dataset['text'][i] for i in val_index]
      val_labels = [dataset['labels'][i] for i in val_index]
     
      # Encode the datasets
      train_encodings = encode_dataset(tokenizer, train_texts, train_labels)
      val_encodings = encode_dataset(tokenizer, val_texts, val_labels)
      



      print(train_encodings)
      #encodings = encode_dataset(tokenizer, dataset['text'], dataset['labels'])
      #train_encodings, train_labels = smote.fit_resample(train_encodings, train_labels)
      #X_resampled, y_resampled = smote.fit_resample(encodings['text'], encodings['labels'])

      # Prepare the training arguments
      training_args = TrainingArguments(
          output_dir='./results',
          num_train_epochs=3,
          per_device_train_batch_size=8,
          per_device_eval_batch_size=8,
          warmup_steps=500,
          weight_decay=0.01,
          logging_dir='./logs',
          logging_steps=10,
          evaluation_strategy="epoch",
      )

      # Define the model
      model = AutoModelForSequenceClassification.from_pretrained(m, num_labels=2)

      # Create the Trainer
      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train_encodings,
          eval_dataset=val_encodings,
          compute_metrics=lambda eval_pred: {
            'accuracy': accuracy_score(eval_pred.label_ids, eval_pred.predictions.argmax(-1)),
            'precision': precision_score(eval_pred.label_ids, eval_pred.predictions.argmax(-1)),
            'recall': recall_score(eval_pred.label_ids, eval_pred.predictions.argmax(-1)),
            'f1': f1_score(eval_pred.label_ids, eval_pred.predictions.argmax(-1)),
          }
      )

      # Train and evaluate the model
      trainer.train()
      eval_metrics = trainer.evaluate()

      # Store the evaluation metrics
      accuracy_list.append(eval_metrics['eval_accuracy'])
      precision_list.append(eval_metrics['eval_precision'])
      recall_list.append(eval_metrics['eval_recall'])
      f1_list.append(eval_metrics['eval_f1'])

      # Save the trained model
      # model_save_path = f"saved_models/model_fold_{fold + 1}"
      # model.save_pretrained(model_save_path)

  # Print the standard evaluation metrics
  print("sampling technique: ", sampling_technique)
  print("Model's Results of ", m)
  print(f"Accuracy: {np.mean(accuracy_list)} (±{np.std(accuracy_list)})")
  print(f"Precision: {np.mean(precision_list)} (±{np.std(precision_list)})")
  print(f"Recall: {np.mean(recall_list)} (±{np.std(recall_list)})")
  print(f"F1-score: {np.mean(f1_list)} (±{np.std(f1_list)})")


unprocessed


<ipython-input-5-6e3c49d82591>:52: FutureWarning: The default value of regex will change from True to False in a future version.
  df['no_spaces'] = df['replaced_author'].str.replace(r'\s+', ' ')
<ipython-input-5-6e3c49d82591>:54: FutureWarning: The default value of regex will change from True to False in a future version.
  df['no_punctuation'] =  df['no_spaces'].str.replace('[{}]'.format(string.punctuation), '')


before sampling
we have 1413 relevant to streamer messages, and 1991 irrelevant to streamer messages
author name was replaced with @author 36.0 times
user name was replaced with @user 99.0 times
After Sampling
we have 1413 relevant to streamer messages, and 1991 irrelevant to streamer messages
681
2723


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.516000,0.524714,0.743025,0.823699,0.714286,0.765101
2,0.504700,0.577182,0.709251,0.873606,0.588972,0.703593
3,0.501800,0.470186,0.791483,0.800937,0.857143,0.828087


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.397000,0.538385,0.745962,0.818697,0.726131,0.769640
2,0.455800,0.541642,0.765051,0.841954,0.736181,0.785523
3,0.132800,0.907143,0.779736,0.829787,0.783920,0.806202


680
2724


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.410800,0.466060,0.755882,0.740664,0.896985,0.811364
2,0.329400,0.453499,0.802941,0.833333,0.829146,0.831234
3,0.092900,0.708152,0.816176,0.864000,0.814070,0.838292


sampling technique:  0
Model's Results of  distilbert-base-uncased
Accuracy: 0.8005333851602314 (±0.014370456164907897)
Precision: 0.846455912558447 (±0.01498131999142143)
Recall: 0.8051221017367538 (±0.012178097920051747)
F1-score: 0.825237209310305 (±0.012459159321526487)


In [ ]:
#poche replication
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

# Load data from CSV file
data = load_and_preprocess_data()
X = data['message']
y = data['label']

# Define the stemmer and stopwords for preprocessing
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

# Define the vectorizers for the three preprocessing methods
vectorizers = [
    ('No Preprocessing', CountVectorizer(), None),
    ('Stemming', CountVectorizer(analyzer=stemmer.stem), stemmer),
    ('Stopwords Removal and Stemming', CountVectorizer(analyzer=stemmer.stem, stop_words=stop_words), stemmer)
]

# Define the classifiers
classifiers = [
    ('Naive Bayes', MultinomialNB()),
    ('SVM', LinearSVC())
]

# Define the evaluation metrics
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Perform 10-fold cross-validation for each preprocessing method and each classifier
for vectorizer_name, vectorizer, stemmer in vectorizers:
    X_processed = X.apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]) if stemmer else x)
    X_processed = vectorizer.fit_transform(X_processed)

    for classifier_name, classifier in classifiers:
        pipeline = make_pipeline(vectorizer, classifier)
        scores = cross_val_score(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True, random_state=42), scoring=metrics)
        print(f"{vectorizer_name} with {classifier_name}:\n"
              f"Accuracy: {scores.mean(axis=0)[0]:.4f}\n"
              f"Precision: {scores.mean(axis=0)[1]:.4f}\n"
              f"Recall: {scores.mean(axis=0)[2]:.4f}\n"
              f"F1-score: {scores.mean(axis=0)[3]:.4f}\n")


In [ ]:
dataset = load_and_preprocess_data()

# initialize the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# define the number of cross-validation folds
num_folds = 5

# define the sampling techniques
sampling_techniques = [
    ('SMOTE', SMOTE()),
    ('TomekLinks', TomekLinks()),
    ('NeighbourhoodCleaningRule', NeighbourhoodCleaningRule())
]

# define the evaluation metrics
metrics = ['accuracy', 'precision', 'recall', 'f1-score']

# define the k-fold cross-validation strategy
skf = StratifiedKFold(n_splits=num_folds, shuffle=True)

# initialize the arrays to store the evaluation results for each fold and each sampling technique
results = {technique[0]: {metric: [] for metric in metrics} for technique in sampling_techniques}

# perform k-fold cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(dataset['text'], dataset['labels'])):
    # extract the training and validation sets for the current fold

   
    X_train = [dataset['text'][i] for i in train_index]
    y_train = [dataset['labels'][i] for i in train_index]
    
    X_train = np.array(X_train)
    X_train = np.reshape(X_train, (-1, 1))
    print(X_train.shape)
    y_train = np.array(y_train)
    y_train = np.reshape(y_train, (-1, 1))
    print(y_train)

    X_val, y_val = [dataset['text'][i] for i in val_index], [dataset['labels'][i] for i in val_index]

    # apply each sampling technique to the training set
    for technique_name, sampler in sampling_techniques:
        # x_train = np.array(X_train).reshape(-1, 1)
        # y_train =  np.array(y_train).reshape(-1, 1)
        #X_val = np.array(X_val).reshape(-1, 1)
        X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)

        # tokenize the resampled training and validation sets
        train_inputs = tokenizer(X_train_resampled.tolist(), padding=True, truncation=True, max_length=512, return_tensors='tf')
        val_inputs = tokenizer(X_val.tolist(), padding=True, truncation=True, max_length=512, return_tensors='tf')

        # train the BERT model on the resampled training set and evaluate it on the validation set
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss=model.compute_loss, metrics=['accuracy'])
        model.fit(train_inputs, y_train_resampled, validation_data=(val_inputs, y_val), batch_size=16, epochs=3)

        # predict the labels for the validation set and compute the evaluation metrics
        y_pred = model.predict(val_inputs, batch_size=16)
        y_pred_classes = np.argmax(y_pred, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred_classes, average='binary')

        # store the evaluation results for the current fold and sampling technique
        results[technique_name]['accuracy'].append(accuracy)
        results[technique_name]['precision'].append(precision)
        results[technique_name]['recall'].append(recall)
        results[technique_name]['f1-score'].append(f1)

# compute the mean evaluation metrics for each sampling technique
for technique_name, _ in sampling_techniques:
    print(f"Mean evaluation metrics for {technique_name}:")
    for metric in metrics:
        mean_metric = np.mean(results[technique_name][metric])
        print(f"{metric}: {mean_metric:.4f}")

# New Section

# New Section